## Import All Required Libraries

In [16]:
# Import all connection libraries

from spotipy import SpotifyClientCredentials
import spotipy
import sqlalchemy as sql
import psycopg2

# Import all data manipulation libraries

import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
pandarallel.initialize(verbose=0, nb_workers=8)

In [17]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

## Load Initial Training Data

In [18]:
# establish connection to Spotify API

cid = 'e5448a8a4fdc4b5d98b44e956d50546d'
secret = '8924c0394d3f49a4a569fc03e891aa1b'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

host_name = 'localhost'
database_name = 'Spotify'
engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/Spotify')

# load training data
new_batch = pd.read_csv('distinct_playlists.csv')[['playlist_uri', 'playlist_name']]
new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

outer = new_batch.merge(db_query, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

new_batch = pd.DataFrame(anti_join)
new_batch = new_batch[0:5]
new_batch.head()

,playlist_uri,playlist_name
21,spotify:playlist:01clVI1RPrF8zHMxrqVIs0,2020 top rap and hiphop hits 2022
34,spotify:playlist:02eILNCPJV0hoYxMUreT0L,He is back
50,spotify:playlist:03oy1RKCWyALsukjlxoJpG,"Reggae, this is AFRiCA"
51,spotify:playlist:03Phtu3e3yY63lHJRro2WH,US drill most popular??
52,spotify:playlist:03PKLp1XC1GdqhzIMWgsBa,Club Music 2022 ?? Best of 2022


## Iterate Through Training Data to Change Grain of Data

In [19]:
load_batch = []
series = new_batch['playlist_uri'].to_dict()
for playlist in tqdm(series.values()):
    try:
        tracks = get_playlist_tracks(playlist)
        if len(tracks) >= 100:
                tracks2 = pd.json_normalize(sp.playlist_tracks(playlist, offset=100), record_path=['items'])
                tracks3 = pd.json_normalize(sp.playlist_tracks(playlist, offset=200), record_path=['items'])
                tracks = pd.concat([tracks, tracks2, tracks3])
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])

                load_batch.append(tracks)
        else:
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])
                load_batch.append(tracks)
    except:
            pass
load_batch = pd.concat(load_batch)
load_batch.shape

100%|██████████| 5/5 [00:04<00:00,  1.23it/s]


(486, 61)

## Upload Training Data to playlist_tracks Table

In [20]:
#load_batch.to_sql('playlist_tracks', engine, if_exists='append')

## Identify All Artists in Every Song

In [21]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks ORDER BY track_uri''', engine)

outer = load_batch.merge(df, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

df = pd.Series(anti_join['track_uri'])
base_list = []
for track in tqdm(df):
    try:
        df2 = pd.json_normalize(sp.track(track), record_path=['artists'])
        df2['track_uri'] = track
        df2 = df2[['name', 'track_uri']]
        base_list.append(df2)
    except:
        pass
df2 = pd.concat(base_list)
df2

 56%|█████▌    | 266/476 [00:29<00:23,  9.02it/s]

In [ ]:
df2['track_uri'].nunique()

486

## Get Row Numbers for Pivoting

In [ ]:
df2['RN'] = df2.groupby("track_uri")["name"].rank(method="first", ascending=True)
df2

,name,track_uri,RN
0,Ranking Dread,spotify:track:3i9IygrKJSZ8cs876xILaZ,1.0
0,Peter Tosh,spotify:track:605ZHoLZx7PtQ8fXgvN10K,1.0
0,Marlon Asher,spotify:track:2zAJvh4y1DBXo7KIqQjumc,1.0
0,Slightly Stoopid,spotify:track:0LQ8OWUZXRd7qu5ujC3758,1.0
0,Bob Marley & The Wailers,spotify:track:5wb91i08i51QP1shXd60ix,1.0
...,...,...,...
1,Badscandal,spotify:track:1XcjDOLpVfQcMIa0RbJdHy,1.0
2,Whoopa,spotify:track:1XcjDOLpVfQcMIa0RbJdHy,3.0
0,Avicii,spotify:track:0nrRP2bk19rLc0orkWPQk2,1.0
0,Swedish House Mafia,spotify:track:2V65y3PX4DkRhy1djlxd9p,2.0


## Pivot DataFrame to Fit all Artists in DataFrame

In [ ]:
df2['artist1'] = np.where(df2['RN'] == 1.0, df2['name'], '')
df2['artist2'] = np.where(df2['RN'] == 2.0, df2['name'], '')
df2['artist3'] = np.where(df2['RN'] == 3.0, df2['name'], '')
df2['artist4'] = np.where(df2['RN'] == 4.0, df2['name'], '')
df2 = df2.drop(columns=['name', 'RN'])
df2

,track_uri,artist1,artist2,artist3,artist4
0,spotify:track:3i9IygrKJSZ8cs876xILaZ,Ranking Dread,,,
0,spotify:track:605ZHoLZx7PtQ8fXgvN10K,Peter Tosh,,,
0,spotify:track:2zAJvh4y1DBXo7KIqQjumc,Marlon Asher,,,
0,spotify:track:0LQ8OWUZXRd7qu5ujC3758,Slightly Stoopid,,,
0,spotify:track:5wb91i08i51QP1shXd60ix,Bob Marley & The Wailers,,,
...,...,...,...,...,...
1,spotify:track:1XcjDOLpVfQcMIa0RbJdHy,Badscandal,,,
2,spotify:track:1XcjDOLpVfQcMIa0RbJdHy,,,Whoopa,
0,spotify:track:0nrRP2bk19rLc0orkWPQk2,Avicii,,,
0,spotify:track:2V65y3PX4DkRhy1djlxd9p,,Swedish House Mafia,,


## Upload All Artist Data to Database

In [ ]:
#df2.to_sql('artists', con=engine, if_exists='append')

## Get All Audio Features

In [ ]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks ORDER BY track_uri''', engine)


outer = load_batch.merge(df, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)
df = pd.Series(anti_join['track_uri'])

base_list = []
for track in tqdm(df):
    try:
        df3 = pd.json_normalize(sp.audio_features(track))
        df3['track_uri'] = track
        df3 = df3[['track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
        base_list.append(df3)
    except:
        pass
df3 = pd.concat(base_list)
df3

100%|██████████| 100/100 [00:08<00:00, 11.65it/s]


,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:001UkMQHw4zXfFNdKpwXAF,0.573,0.846,7,-4.866,1,0.0344,0.003720,0.000079,0.2900,0.562,127.061,191053,4
0,spotify:track:007ibBncyX6qXsy3sSCpgV,0.774,0.963,1,-2.428,1,0.0393,0.065100,0.140000,0.2410,0.974,132.044,340088,4
0,spotify:track:00gscXjMt09aOxodmaDeR4,0.732,0.798,11,-6.511,0,0.0956,0.026900,0.000005,0.0718,0.911,122.008,188918,4
0,spotify:track:00KyYtT6NaXwbPecina5Pj,0.535,0.859,6,-2.922,1,0.0344,0.052800,0.000000,0.3200,0.698,102.008,166507,4
0,spotify:track:00Mb3DuaIH1kjrwOku9CGU,0.487,0.900,0,-4.417,1,0.0482,0.000068,0.000000,0.3580,0.484,149.937,204000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,spotify:track:0a4kXlqLISIf5PSiyp0Zmp,0.748,0.771,6,-4.156,1,0.1500,0.101000,0.000000,0.1350,0.792,142.086,195058,4
0,spotify:track:0a4MMyCrzT0En247IhqZbD,0.593,0.935,4,-3.430,0,0.2250,0.268000,0.000000,0.2860,0.783,99.976,179027,4
0,spotify:track:0a4sEwtxPbqXYmQvEBqQJt,0.927,0.638,8,-5.669,1,0.1020,0.307000,0.000006,0.0691,0.819,131.011,165733,4
0,spotify:track:0a7dqjPE3kZrjYFXFI6ORR,0.644,0.736,9,-6.060,1,0.0362,0.068000,0.000000,0.0962,0.697,105.957,172547,4


## Upload Audio Features to Database

In [ ]:
#df3.to_sql('audio_features', con=engine, if_exists='append')

In [ ]:
all_data = pd.read_sql('''SELECT DISTINCT added_at, is_local, primary_color_x, "added_by.external_urls.spotify", "added_by.href",
"added_by.id", "added_by.type", "added_by.uri", "track.album.album_type", "track.album.external_urls.spotify", 
"track.album.href", "track.album.id", album_name, release_date, "track.album.release_date_precision", 
"track.album.total_tracks", "track.album.type", album_uri, "track.disc_number", pt.duration_ms, "track.episode",
"isExplicit", "track.external_ids.isrc", "track.external_urls.spotify", "tracks.href", "track.id", "track.is_local", 
track_name, "track.popularity", "track.preview_url", "track.track", "track.track_number", "track.type", pt.track_uri, 
"video_thumbnail.url", playlist_uri, collaborative, description, href, id, playlist_name, primary_color_y,
public, pt.snapshot_id, type, uri, "external_urls.spotify", "followers.href", "followers.total", "owner.display_name",
"owner.external_urls.spotify", "owner.href", "owner.id", "owner.type", "owner.uri", "tracks.href", "tracks.limit", "tracks.next", 
"tracks.offset", "tracks.previous", "tracks.total", track, video_thumbnail, danceability, energy, key, loudness, mode,
speechiness, acousticness, instrumentalness, liveness, valence, tempo, time_signature
FROM playlist_tracks pt
JOIN audio_features af ON pt.track_uri = af.track_uri''', engine)
f = open("s3.csv", "w")
f.truncate()
f.close()

#all_data.to_csv('s3.csv', mode="w+")